In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
import scipy.stats as stats


In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
df=pd.read_csv('stud.csv')

In [5]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [6]:
df.tail()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77
999,female,group D,some college,free/reduced,none,77,86,86


In [9]:
df.shape

(1000, 8)

In [17]:
x=df.drop(columns=['math_score'],axis=1)

In [31]:
x.shape

(1000, 19)

In [19]:
y=df['math_score']

In [1]:
# Create Column Transformer with 3 types of transformers

num_features = x.select_dtypes(exclude="object").columns


cat_features = x.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),        
    ]
)
#hello this is a test


NameError: name 'x' is not defined

In [28]:
x=preprocessor.fit_transform(x)

In [30]:
x.shape

(1000, 19)

train est test de la data

In [46]:
x_train ,x_test ,y_train ,y_test =train_test_split(x,y,test_size=0.2,random_state=42)
x_train.shape, x_test.shape

((800, 19), (200, 19))

In [48]:
def evaluate_model(true,predict):
    mae=mean_absolute_error(true,predict)
    mse=mean_squared_error(true,predict)
    r2=r2_score(true,predict)
    rmse=np.sqrt(mse)
    return mae , mse , r2 ,rmse

In [ ]:
models={
    "linear regression":LinearRegression(),
    "Decision Tree":DecisionTreeClassifier(),
    "KNN":KNeighborsClassifier(),

}

model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    #make predctions
y_train_pred=model.predict(x_train)
y_test_pred=model.predict(x_test)



model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
 model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)




NameError: name 'evaluate_model' is not defined